In [1]:
from config import config
config = config.Configuration()
run_prev_notebooks = False

In [2]:
import tensorflow as tf

import numpy as np
import rasterio
# from rasterio.windows import Window

import os
import time
import rasterio.warp # Reproject raster samples
from functools import reduce
from tensorflow.keras.models import load_model

from core.losses import tversky, accuracy, dice_coef, dice_loss, specificity, sensitivity
from core.optimizers import adaDelta #, adagrad, adam, nadam

import json
from sklearn.model_selection import train_test_split

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Polygon

import warnings # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

define_model()# Pipeline

In [3]:
def data_preparation():
    print("1: Data_preparation") 
    %run "1_data_preparation.ipynb"
def define_model():
    print("2: Model")
    %run "2_model.ipynb"
def training():
    print("3: Training")
    %run "3_training.ipynb"
def evaluation():
    print("4: Evaluation")
    %run "4_evaluation.ipynb"
def prediction():
    print("5: Prediction")
    %run "5_prediction.ipynb"
def data_cleanup():
    print("6: Data Cleanup")
    %run "6_data_cleanup.ipynb"

In [4]:
data_preparation()
define_model()

1: Data_preparation
2: Model
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['Input[0][0]']                  
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 64  36928       ['conv2d[0][0]']                 
                                )                                

GPU Information

In [5]:
import torch
use_cuda = torch.cuda.is_available()

In [6]:
if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

__CUDNN VERSION: 8005
__Number CUDA Devices: 3
__CUDA Device Name: GeForce RTX 3090
__CUDA Device Total Memory [GB]: 25.447170048


In [7]:
print(f"GPU is available: {tf.test.is_gpu_available()}. List of GPUs: {tf.config.list_physical_devices('GPU')}")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available: False. List of GPUs: []


In [8]:
device = torch.device("cuda" if use_cuda else "cpu")
# device = torch.device("cuda:2" if use_cuda else "cpu")
print("Device: ",device)

Device:  cuda


Prepare Data for GPU Training

In [9]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

print(tf.data.experimental.cardinality(train_ds))
print(tf.data.experimental.cardinality(val_ds))
print(tf.data.experimental.cardinality(test_ds))

tf.Tensor(2349, shape=(), dtype=int64)
tf.Tensor(588, shape=(), dtype=int64)
tf.Tensor(735, shape=(), dtype=int64)


In [10]:
X_train_t = torch.FloatTensor(X_train)
y_train_t = torch.FloatTensor(y_train)

X_test_t = torch.FloatTensor(X_test)
y_test_t = torch.FloatTensor(y_test)

X_val_t = torch.FloatTensor(X_val)
y_val_t = torch.FloatTensor(y_val)

Move Data to GPU

In [15]:
X_train, y_train = torch.tensor(X_train).to(device), torch.tensor(y_train).to(device)

In [22]:
import torch.nn as nn
import torch.nn.functional as F
from core.UNet_class import UNetModel

class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        # Some init for my module
        
    def forward(self, x):
        # Function for performing the calculation of the module.
        pass

shape_input = tf.convert_to_tensor(np.array([config.BATCH_SIZE, *config.input_shape]))
model = UNetModel(shape_input, [config.input_shape[-1]])
#shape_input = torch.tensor(np.array([config.BATCH_SIZE, *config.input_shape]))
#tf.convert_to_tensor
#model = UNetModel(shape_input,config.input_label_channel)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_20 (Conv2D)             (None, 256, 256, 64  1792        ['Input[0][0]']                  
                                )                                                                 
                                                                                                  
 conv2d_21 (Conv2D)             (None, 256, 256, 64  36928       ['conv2d_20[0][0]']              
                                )                                                           

In [29]:
model = model.to(device=device)

In [37]:
print(f"Trainingsdata is on: {X_train.device} - {X_train.is_cuda}")
# print(f"Model is on: {model.device} - {model.is_cuda}")

Trainingsdata is on: cuda:1 - True


In [51]:
from core.losses import tversky, accuracy, dice_coef, dice_loss, specificity, sensitivity
from core.optimizers import adaDelta
from functools import reduce


OPTIMIZER = adaDelta
LOSS = tversky 

#Only for the name of the model in the very end
OPTIMIZER_NAME = 'AdaDelta'
LOSS_NAME = 'weightmap_tversky'

In [52]:
def fit(epochs=config.NB_EPOCHS,lr=0.001,model=model,train_loader=train_ds,val_loader=val_ds,opt_func=OPTIMIZER, batch_size=config.BATCH_SIZE):
    history = []
    optimizer = opt_func
    for epoch in range(epochs):
        #training phase
        for batch in range(batch_size):
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        #Validation phase
        result = evaluate(model,val_loader)
        model.epoch_end(epoch,result)# Result after each epoch
        history.append(result)
    return history
def evaluate(model,val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [53]:
fit(train_loader=train_ds,
    epochs=config.NB_EPOCHS,
    lr=0.001,
    model=model,
    val_loader=val_ds,
    #steps_per_epoch=config.MAX_TRAIN_STEPS,
    # validation_steps=config.VALID_IMG_COUNT,
    batch_size=config.BATCH_SIZE,
    #callbacks=callbacks_list
    # workers=1,
    # use_multiprocessing=True # the generator is not very thread safe
)

TypeError: 'int' object is not iterable

In [32]:
# %%time
# shape incompatible
# with torch.no_grad():
# training()
loss_history = [model.fit(train_ds,
                        epochs=config.NB_EPOCHS, 
                        validation_data=val_ds,
                        steps_per_epoch=config.MAX_TRAIN_STEPS,
                        # validation_steps=config.VALID_IMG_COUNT,
                        batch_size=config.BATCH_SIZE,
                        callbacks=callbacks_list
                        # workers=1,
                        # use_multiprocessing=True # the generator is not very thread safe
                )]
print("Training completed.")

ModuleAttributeError: 'UNetModel' object has no attribute 'fit'

In [ ]:
def train_model(model, optimizer, data_loader, loss_module, num_epochs=config.NB_EPOCHS):
    # Set model to train mode
    model.train() 
    
    # Training loop
    for epoch in tqdm(range(num_epochs)):
        for data_inputs, data_labels in data_loader:
            
            ## Step 1: Move input data to device (only strictly necessary if we use GPU)
            data_inputs = data_inputs.to(device)
            data_labels = data_labels.to(device)
            
            ## Step 2: Run the model on the input data
            preds = model(data_inputs)
            preds = preds.squeeze(dim=1) # Output is [Batch size, 1], but we want [Batch size]
            
            ## Step 3: Calculate the loss
            loss = loss_module(preds, data_labels.float())
            
            ## Step 4: Perform backpropagation
            # Before calculating the gradients, we need to ensure that they are all zero. 
            # The gradients would not be overwritten, but actually added to the existing ones.
            optimizer.zero_grad() 
            # Perform backpropagation
            loss.backward()
            
            ## Step 5: Update the parameters
            optimizer.step()

In [ ]:
train_model(model, optimizer, train_data_loader, loss_module)

In [ ]:
training()

In [40]:
# pip install tensorflow-gpu
print(tf.config.list_physical_devices('GPU'))

tf.debugging.set_log_device_placement(True)

try:
  # Specify an invalid GPU device
  with tf.device('/device:GPU:2'):
    
except RuntimeError as e:
  print(e)

[]


In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ("data_preparation", data_preparation()),
    ("model", define_model()),
    ("training", training()),
    ("evaluation", evaluation()),
    # ("prediction", prediction()),
    # ("data_cleanup", data_cleanup())
])

- Putting Everything Together (cf. p. 553 & davor)
    - Chaining Transformations
    - shuffle data etc.
- Präsi: Loading Preprocessing Data with TensorFlow